# SQL-first Analytics approach with SQLite and Pandas
A series of SQL queries to perform:
- Basic aggregations
- Time-series growth analysis
- Segment profitability and ranking

In [1]:
import sqlite3
import pandas as pd

## Loaded it into an SQLite in-memory database for SQL querying

In [14]:

financial_data = pd.read_csv('Financial_Sample_Cleaned.csv')
targets = pd.read_csv('monthly_targets.csv')

# Create an in-memory SQLite database and load the DataFrame as a SQL table
conn = sqlite3.connect(':memory:')
financial_data.to_sql('financial_data', conn, index=False, if_exists='replace')

718

## Revenue & Profit Aggregation by Segment
Calculated total revenue, average profit, and transaction count for each customer segment.

In [18]:
# Define and execute the SQL queries

# Basic aggregation
query = """
SELECT 
    Segment,
    SUM(Sales) as total_revenue,
    AVG(Profit) as avg_profit,
    COUNT(*) as transactions
FROM financial_data 
GROUP BY Segment
ORDER BY total_revenue DESC;
"""
result = pd.read_sql_query(query, conn)
print("Basic Aggregation:")
print(result)

Basic Aggregation:
            Segment  total_revenue    avg_profit  transactions
0        Government   5.250426e+07  37960.577233           300
1    Small Business   4.242792e+07  41431.685000           100
2        Enterprise   1.961169e+07  -6145.456250           100
3         Midmarket   2.381883e+06   6601.030750           100
4  Channel Partners   1.800594e+06  13168.031400           100
5              None            NaN           NaN            18


## Monthly Sales Growth Analysis
Used window functions to compute month-over-month sales growth rates.


In [19]:
# Window functions (SQLite supports window functions from version 3.25+)
query = """
SELECT 
    Month_Name,
    SUM(Sales) as monthly_sales,
    LAG(SUM(Sales)) OVER (ORDER BY Month_Number) as prev_month,
    (SUM(Sales) - LAG(SUM(Sales)) OVER (ORDER BY Month_Number)) * 100.0 / 
    LAG(SUM(Sales)) OVER (ORDER BY Month_Number) as growth_rate
FROM financial_data
GROUP BY Month_Number, Month_Name
ORDER BY Month_Number;
"""
result = pd.read_sql_query(query, conn)
print("\nWindow Functions:")
print(result)


Window Functions:
   Month_Name  monthly_sales   prev_month  growth_rate
0        None            NaN          NaN          NaN
1     January     6607761.68          NaN          NaN
2    February     7297531.39   6607761.68    10.438780
3       March     5586859.87   7297531.39   -23.441784
4       April     6964775.07   5586859.87    24.663500
5         May     6210211.06   6964775.07   -10.834004
6        June     9518893.82   6210211.06    53.278105
7        July     8102920.18   9518893.82   -14.875401
8      August     5864622.42   8102920.18   -27.623347
9   September    10882697.27   5864622.42    85.565182
10    October    21671431.02  10882697.27    99.136579
11   November    12651417.50  21671431.02   -41.621679
12   December    17367228.98  12651417.50    37.274965


## Segment Profitability & Ranking
Leveraged a Common Table Expression to summarize revenue, costs, and profit, then assigned a rank by profitability to each segment.

In [20]:
# CTEs and window functions
query = """
WITH segment_performance AS (
    SELECT 
        Segment,
        SUM(Sales) as revenue,
        SUM(COGS) as costs,
        SUM(Profit) as profit
    FROM financial_data
    GROUP BY Segment
)
SELECT 
    Segment,
    revenue,
    profit,
    ROUND(profit*100.0/revenue, 2) as profit_margin,
    ROW_NUMBER() OVER (ORDER BY profit DESC) as profit_rank
FROM segment_performance;
"""
result = pd.read_sql_query(query, conn)
print("\nCTEs and Ranking:")
print(result)



CTEs and Ranking:
            Segment       revenue        profit  profit_margin  profit_rank
0        Government  5.250426e+07  1.138817e+07          21.69            1
1    Small Business  4.242792e+07  4.143168e+06           9.77            2
2  Channel Partners  1.800594e+06  1.316803e+06          73.13            3
3         Midmarket  2.381883e+06  6.601031e+05          27.71            4
4        Enterprise  1.961169e+07 -6.145456e+05          -3.13            5
5              None           NaN           NaN            NaN            6


## Customer Risk & Value Segmentation
Built a multi-step analysis that:
1. Groups customers by segment & country  
2. Calculates frequency, avg transaction, total value, volatility, and NTILE quartiles  
3. Classifies risk category (VIP, High Value, High Risk, Standard)

In [11]:
# Customer Risk Analysis SQL query
query_customer_risk = """
WITH customer_risk AS (
    SELECT 
        Segment,
        Country,
        COUNT(*) as transaction_frequency,
        AVG(Sales) as avg_transaction,
        SUM(Sales) as total_value,
        MAX(Sales) - MIN(Sales) as value_volatility,
        NTILE(5) OVER (ORDER BY SUM(Sales) DESC) as value_quintile
    FROM financial_data
    GROUP BY Segment, Country
)
SELECT 
    *,
    CASE 
        WHEN value_quintile = 5 AND transaction_frequency > 10 THEN 'VIP'
        WHEN value_quintile >= 4 THEN 'High Value'  
        WHEN value_volatility > avg_transaction * 3 THEN 'High Risk'
        ELSE 'Standard'
    END as risk_category
FROM customer_risk;
"""

customer_risk_result = pd.read_sql_query(query_customer_risk, conn)
print("Customer Risk Analysis:")
print(customer_risk_result)


Customer Risk Analysis:
             Segment                   Country  transaction_frequency  \
0         Government                    France                     60   
1     Small Business  United States of America                     20   
2         Government                   Germany                     60   
3         Government                    Canada                     60   
4         Government                    Mexico                     60   
5     Small Business                    Canada                     20   
6         Government  United States of America                     60   
7     Small Business                    France                     20   
8     Small Business                   Germany                     20   
9     Small Business                    Mexico                     20   
10        Enterprise  United States of America                     20   
11        Enterprise                   Germany                     20   
12        Enterprise       

In [9]:
# Close connection when done
conn.close()
